# Preparing a dataset

Input:  is raw dataset with tick dataset

Output: Volume bars dataset with cumulative percentage sum on each day

In [1]:
import mydatalabeling as mdl
import myplotlib as mpl

import pandas as pd
import gc

In [2]:
#input file with ticks data
infp = "IVE_tickbidask.txt"

#read 
df = mdl.read_ticks(infp)
df = df.rename(columns={"size":"v"})
#calculate spread
df['spread'] = df.ask - df.bid

#only main session
f = (df.time > '09:30:00') & (df.time < '16:00:00') & (df.index < '2018-01-01') & (df.index > '2017-01-01')
df = df[f]
display(df.head(1))
display(df.tail(1))

,time,price,bid,ask,v,spread
date,,,,,,
2017-01-03,09:30:08,102.2,102.09,102.2,400,0.11


,time,price,bid,ask,v,spread
date,,,,,,
2017-12-29,15:59:59,114.215,114.22,114.23,100,0.01


### ---------------
Here we will tranform our price to cumulative percentage 
change and then generate volume bars
### ---------------

In [3]:
%%time
#calculate cusum for everyday
df = mdl.all_value_to_cusum__mp(df, 'price')

# select bar size in volume
volume_M = 1000

#generate volumebars
vdf,tick_df = mdl.generate_all_vloume_bars_mp(df, volume_M)

del tick_df
vdf.head(2)

CPU times: user 9.56 s, sys: 303 ms, total: 9.87 s
Wall time: 1min 6s


,Nr,open,high,low,close,sum_vol,time
date,,,,,,,
2017-01-03,0,0.0000,0.00,-0.0100,0.0,1232,09:30:08
2017-01-03,5,-0.0195,0.04,-0.0195,0.0,1062,09:30:15


In [4]:
#plot one day
mpl.plotChartWithSignal(vdf.reset_index(),'2017-06-07', show_only_bars = True)

In [5]:
#store our dataset to the pickle file
vdf.to_pickle('final_ds_cusum_small.pkl')

#delete everything
del vdf
del df
gc.collect()

15104